In [3]:
from joblib import Parallel, delayed
from library.utils import evaluate, read_data
from sklearn.svm import SVC
from library.configs import SCORERS, CV

In [2]:
X,yn,yr = read_data("groovy-1_5_7.csv",True)

groovy-1_5_7.csv noise:0.085, imb:8.463,80,677, Shape:(757, 65)


In [8]:
%%time
r = evaluate(SVC(probability=True,random_state=42),X,yn,yr,CV,SCORERS)

CPU times: user 1.52 s, sys: 14 µs, total: 1.52 s
Wall time: 1.52 s


In [10]:
p = Parallel(n_jobs=4)
f = delayed(func)
p(f)

defaultdict(list,
            {'matthews_corrcoef': array([0.27215318, 0.69748583, 0.70867445, 0.14509261, 0.66090312,
                    0.44125048, 0.70237692, 0.45357784, 0.6001315 , 0.55731682,
                    0.74324324, 0.34385097, 0.47931034, 0.38868386, 0.66090312,
                    0.51342707, 0.5574549 , 0.48104086, 0.31981982, 0.55203931,
                    0.47945205, 0.58639456, 0.55203931, 0.48104086, 0.48639456]),
             'pr_rec_score': array([0.06905878, 0.66281032, 0.84054233, 0.0741302 , 0.67873162,
                    0.36366857, 0.62094847, 0.16343123, 0.63170796, 0.54832284,
                    0.75622467, 0.42408757, 0.34139367, 0.48679029, 0.72690131,
                    0.62698413, 0.60433728, 0.50713503, 0.16858077, 0.53796903,
                    0.52788963, 0.4116863 , 0.53672094, 0.22643571, 0.50111155])})

In [ ]:
    scores = defaultdict(list)
    for train_id, test_id in cv.split(X,y_noisy):  #vs y_noisy, to solve no-pos-label-in-test-set bug - Changed again
        #print(np.unique(y_noisy[train_id],return_counts=True)[1])
        try:
            clf = clf.fit(X[train_id],y_noisy[train_id])
        except Exception as e:
            print("ERROR!!!",str(e))
            continue
        probs = clf.predict_proba(X[test_id])
        for func in scorers:
            try:
                func([0,1,1],[.2,.6,.7])
                yp = probs[:,1]
            except ValueError as e:
                yp = np.argmax(probs,axis=1)
            scores[func.__name__].append(func(y_real[test_id],yp))
    
    for func in scorers:
        if len(scores[func.__name__])>0:
            scores[func.__name__] = np.array(scores[func.__name__])
        else: 
            scores[func.__name__] = np.array([0.0]) # Training failed in every fold        
    return scores